In [24]:
# Taken from: 
# https://github.com/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018/Stock_NeurIPS2018_1_Data.ipynb

In [1]:
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-j393302m
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-j393302m
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 0496601ee2824ca675beb5a220afd9109ac02ffa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-vtixvy97/elegantrl_3ec3bdf4763d4055b9405968e05b46f7
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-vtixvy97/elegantrl_3ec3bdf4763d4055b9405968e05b46f7
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 5bca7aeeb73f5779788dd6714a53b59ce880c583
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

2024-11-20 14:39:13.153176: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 14:39:13.170544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732088353.192265   76911 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732088353.198940   76911 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 14:39:13.221302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [4]:
aapl_df_yf.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2020-01-02 00:00:00+00:00,72.796021,75.087502,75.150002,73.797501,74.059998,135480400
2020-01-03 00:00:00+00:00,72.088303,74.357498,75.144997,74.125000,74.287498,146322800
2020-01-06 00:00:00+00:00,72.662720,74.949997,74.989998,73.187500,73.447502,118387200
2020-01-07 00:00:00+00:00,72.320969,74.597504,75.224998,74.370003,74.959999,108872000
2020-01-08 00:00:00+00:00,73.484344,75.797501,76.110001,74.290001,74.290001,132079200


In [5]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (20, 8)


In [6]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,72.796021,75.087502,75.150002,74.059998,135480400,aapl,3
1,2020-01-03,72.088303,74.357498,75.144997,74.287498,146322800,aapl,4
2,2020-01-06,72.662720,74.949997,74.989998,73.447502,118387200,aapl,0
3,2020-01-07,72.320969,74.597504,75.224998,74.959999,108872000,aapl,1
4,2020-01-08,73.484344,75.797501,76.110001,74.290001,132079200,aapl,2


In [7]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [8]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [9]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (94301, 8)


In [10]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,2.733996,3.241071,3.251429,3.067143,746015200,AAPL,4
1,2009-01-02,41.484413,58.990002,59.080002,58.590000,6547900,AMGN,4
2,2009-01-02,15.014070,19.330000,19.520000,18.570000,10955700,AXP,4
3,2009-01-02,33.941097,45.250000,45.560001,42.799999,7010200,BA,4
4,2009-01-02,30.602571,46.910000,46.980000,44.910000,7117200,CAT,4


In [11]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (3228, 8)


Successfully added vix
Successfully added turbulence index


In [12]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [13]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.733996,3.241071,3.251429,3.067143,746015200.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,3.067143,3.067143,39.580002,0.0
1,2009-01-02,AMGN,41.484413,58.990002,59.080002,58.590000,6547900.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,58.590000,58.590000,39.580002,0.0
2,2009-01-02,AXP,15.014070,19.330000,19.520000,18.570000,10955700.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,18.570000,18.570000,39.580002,0.0
3,2009-01-02,BA,33.941097,45.250000,45.560001,42.799999,7010200.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,42.799999,42.799999,39.580002,0.0
4,2009-01-02,CAT,30.602571,46.910000,46.980000,44.910000,7117200.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,44.910000,44.910000,39.580002,0.0


In [16]:
len(list_ticker)

29

In [15]:
len(list_date)

4682

In [17]:
len(processed_full)

93612

In [18]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
9715


In [20]:
import os
os.makedirs("data", exist_ok=True)

In [22]:
train.to_csv('data/train_data.csv', index=False)
trade.to_csv('data/trade_data.csv', index=False)